In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
importlib.reload(ezpv)

In [ ]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev
import pandas as pd

In [ ]:
# For computation
import numpy as np
import math
import scipy.stats as stats
import matplotlib.pyplot as plt

In [ ]:
import geopandas as gpd
import shapely as shp

In [ ]:
DATASTORE_URL = "http://cardshark.cs.berkeley.edu"
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
sd_la = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "unimodal_trip_car_bike_mtv_la")
sd_sj = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "car_scooter_brex_san_jose")
sd_ucb = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "train_bus_ebike_mtv_ucb")

In [ ]:
import importlib
importlib.reload(ezpv)

In [ ]:
pv_la = eipv.PhoneView(sd_la)

In [ ]:
pv_sj = eipv.PhoneView(sd_sj)

In [ ]:
pv_ucb = eipv.PhoneView(sd_ucb)

### Experimenting with creating temporal reference trajectories

#### `shapely.snap`: Easy case (max spatial error for accuracy control < 20 m)

works pretty well

In [ ]:
android_trip = pv_la.map()["android"]["ucb-sdb-android-1"]["evaluation_ranges"][1]["evaluation_trip_ranges"][0]
ios_trip = pv_la.map()["ios"]["ucb-sdb-ios-1"]["evaluation_ranges"][1]["evaluation_trip_ranges"][0]
android_accuracy_df = android_trip["location_df"]
ios_accuracy_df = ios_trip["location_df"]

In [ ]:
android_control_gpdf = gpd.GeoDataFrame(
        android_accuracy_df, geometry=android_accuracy_df.apply(lambda lr: shp.geometry.Point(lr.longitude, lr.latitude), axis=1))
ios_control_gpdf = gpd.GeoDataFrame(
        ios_accuracy_df, geometry=ios_accuracy_df.apply(lambda lr: shp.geometry.Point(lr.longitude, lr.latitude), axis=1))

In [ ]:
len(android_control_gpdf), len(ios_control_gpdf)

In [ ]:
snapped_result = shp.ops.snap(shp.geometry.LineString(coordinates=list(zip(android_control_gpdf.longitude, android_control_gpdf.latitude))),
             shp.geometry.LineString(coordinates=list(zip(ios_control_gpdf.longitude, ios_control_gpdf.latitude))),
            tolerance=TEN_METERS_IN_DEGREES); snapped_result

This looks good in this small view, but looking at it in a map, we can see some weirdnesses at the end and at the curve on Miramonte

In [ ]:
import folium

In [ ]:
curr_map = folium.Map()
gt_leg_gj = sd_ucb.get_geojson_for_leg(gt_leg)
sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(snapped_result.coords)), properties={"style": {"color": "red"}})
old_sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(zip(android_control_gpdf.longitude, android_control_gpdf.latitude))),properties={"style": {"color": "yellow"}, "ts": list(android_control_gpdf.ts)})
filtered_sensed_section_gj = sensed_section_gj
gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj[2], name="ground_truth_points", color="green")
sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values")
old_sensed_leg_gj_feature = folium.GeoJson(old_sensed_section_gj, name="old sensed_values")
filtered_sensed_leg_gj_feature = folium.GeoJson(filtered_sensed_section_gj, name="filtered sensed_values")
sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="sensed_points", color="red", tz="America/Los_Angeles")
old_sensed_leg_gj_points = ezpv.get_point_markers(old_sensed_section_gj, name="old_sensed_points", color="yellow", tz="America/Los_Angeles")
filtered_sensed_leg_gj_points = ezpv.get_point_markers(filtered_sensed_section_gj, name="filtered_sensed_points", color="blue", tz="America/Los_Angeles")
curr_map.add_child(gt_leg_gj_feature)
curr_map.add_child(gt_leg_gj_points)
curr_map.add_child(sensed_leg_gj_feature)
curr_map.add_child(sensed_leg_gj_points)
curr_map.add_child(old_sensed_leg_gj_feature)
curr_map.add_child(old_sensed_leg_gj_points)
curr_map.add_child(filtered_sensed_leg_gj_feature)
curr_map.add_child(filtered_sensed_leg_gj_points)
curr_map.fit_bounds(sensed_leg_gj_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map

#### `shapely.snap`: Hard case (android accuracy is just completely messed up)

does not work well. And we don't have the distances to do any kind of filtering

In [ ]:
android_trip = pv_la.map()["android"]["ucb-sdb-android-1"]["evaluation_ranges"][0]["evaluation_trip_ranges"][0]
ios_trip = pv_la.map()["ios"]["ucb-sdb-ios-1"]["evaluation_ranges"][0]["evaluation_trip_ranges"][0]
android_accuracy_df = android_trip["location_df"]
ios_accuracy_df = ios_trip["location_df"]

In [ ]:
android_control_gpdf = gpd.GeoDataFrame(
        android_accuracy_df, geometry=android_accuracy_df.apply(lambda lr: shp.geometry.Point(lr.longitude, lr.latitude), axis=1))
ios_control_gpdf = gpd.GeoDataFrame(
        ios_accuracy_df, geometry=ios_accuracy_df.apply(lambda lr: shp.geometry.Point(lr.longitude, lr.latitude), axis=1))

In [ ]:
len(android_control_gpdf), len(ios_control_gpdf)

In [ ]:
snapped_result = shp.ops.snap(shp.geometry.LineString(coordinates=list(zip(android_control_gpdf.longitude, android_control_gpdf.latitude))),
             shp.geometry.LineString(coordinates=list(zip(ios_control_gpdf.longitude, ios_control_gpdf.latitude))),
            tolerance=TEN_METERS_IN_DEGREES); snapped_result

This looks good in this small view, but looking at it in a map, we can see some weirdnesses at the end and at the curve on Miramonte

In [ ]:
import folium

In [ ]:
curr_map = folium.Map()
gt_leg_gj = sd_ucb.get_geojson_for_leg(gt_leg)
sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(snapped_result.coords)), properties={"style": {"color": "red"}})
old_sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(zip(android_control_gpdf.longitude, android_control_gpdf.latitude))),properties={"style": {"color": "yellow"}, "ts": list(android_control_gpdf.ts)})
# filtered_sensed_section_gj = sensed_section_gj
gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj[2], name="ground_truth_points", color="green")
sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values")
old_sensed_leg_gj_feature = folium.GeoJson(old_sensed_section_gj, name="old sensed_values")
# filtered_sensed_leg_gj_feature = folium.GeoJson(filtered_sensed_section_gj, name="filtered sensed_values")
sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="sensed_points", color="red", tz="America/Los_Angeles")
old_sensed_leg_gj_points = ezpv.get_point_markers(old_sensed_section_gj, name="old_sensed_points", color="yellow", tz="America/Los_Angeles")
# filtered_sensed_leg_gj_points = ezpv.get_point_markers(filtered_sensed_section_gj, name="filtered_sensed_points", color="blue", tz="America/Los_Angeles")
curr_map.add_child(gt_leg_gj_feature)
curr_map.add_child(gt_leg_gj_points)
curr_map.add_child(sensed_leg_gj_feature)
curr_map.add_child(sensed_leg_gj_points)
curr_map.add_child(old_sensed_leg_gj_feature)
curr_map.add_child(old_sensed_leg_gj_points)
# curr_map.add_child(filtered_sensed_leg_gj_feature)
# curr_map.add_child(filtered_sensed_leg_gj_points)
curr_map.fit_bounds(sensed_leg_gj_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map

####  `shapely.interp1d`: Easy case

Works pretty well (97.25% retained)

In [ ]:
android_trip = pv_la.map()["android"]["ucb-sdb-android-1"]["evaluation_ranges"][1]["evaluation_trip_ranges"][0]
ios_trip = pv_la.map()["ios"]["ucb-sdb-ios-1"]["evaluation_ranges"][1]["evaluation_trip_ranges"][0]
android_accuracy_df = android_trip["location_df"]
ios_accuracy_df = ios_trip["location_df"]

In [ ]:
android_control_gpdf = gpd.GeoDataFrame(
        android_accuracy_df, geometry=android_accuracy_df.apply(lambda lr: shp.geometry.Point(lr.longitude, lr.latitude), axis=1))
ios_control_gpdf = gpd.GeoDataFrame(
        ios_accuracy_df, geometry=ios_accuracy_df.apply(lambda lr: shp.geometry.Point(lr.longitude, lr.latitude), axis=1))

In [ ]:
len(android_control_gpdf), len(ios_control_gpdf)

In [ ]:
import scipy.interpolate as sci

In [ ]:
android_lat_fn = sci.interp1d(x=android_control_gpdf.ts, y=android_control_gpdf.latitude, fill_value="extrapolate")
android_lon_fn = sci.interp1d(x=android_control_gpdf.ts, y=android_control_gpdf.longitude, fill_value="extrapolate")
ios_lat_fn = sci.interp1d(x=ios_control_gpdf.ts, y=ios_control_gpdf.latitude, fill_value="extrapolate")
ios_lon_fn = sci.interp1d(x=ios_control_gpdf.ts, y=ios_control_gpdf.longitude, fill_value="extrapolate")

In [ ]:
new_ts_range = np.arange(android_trip["start_ts"], android_trip["end_ts"], 1)
new_fmt_time_range = [arrow.get(ts).to(sd_la.eval_tz) for ts in new_ts_range]

In [ ]:
new_android_lat = android_lat_fn(new_ts_range)
new_android_lng = android_lon_fn(new_ts_range)
new_ios_lat = ios_lat_fn(new_ts_range)
new_ios_lng = ios_lon_fn(new_ts_range)
new_android_gpdf = gpd.GeoDataFrame({
    "latitude": new_android_lat,
    "longitude": new_android_lng,
    "ts": new_ts_range,
    "fmt_time": new_fmt_time_range,
    "geometry": [shp.geometry.Point(x, y) for x, y in zip(new_android_lng, new_android_lat)]
})
new_ios_gpdf = gpd.GeoDataFrame({
    "latitude": new_ios_lat,
    "longitude": new_ios_lng,
    "ts": new_ts_range,
    "fmt_time": new_fmt_time_range,
    "geometry": [shp.geometry.Point(x, y) for x, y in zip(new_ios_lng, new_ios_lat)]
})

In [ ]:
new_android_gpdf[["fmt_time", "geometry"]].head(), new_ios_gpdf[["fmt_time", "geometry"]].head()

In [ ]:
distances = new_android_gpdf.distance(new_ios_gpdf) * (R/360); distances.head()

In [ ]:
valid_mask = distances <= 2; np.nonzero(np.logical_not(valid_mask))

In [ ]:
filtered_android_gpdf = new_android_gpdf[valid_mask]; filtered_android_gpdf.head()
filtered_ios_gpdf = new_ios_gpdf[valid_mask]; filtered_ios_gpdf.head()

In [ ]:
valid_ratio = len(filtered_android_gpdf)/len(new_android_gpdf)
assert valid_ratio == len(filtered_ios_gpdf)/len(new_ios_gpdf)
valid_ratio

In [ ]:
ifig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,15))
# android_control_gpdf.plot(ax=ax)
# ios_control_gpdf.plot(ax=ax)
new_android_gpdf.plot(ax=ax)
new_ios_gpdf.plot(ax=ax)
filtered_android_gpdf.plot(ax=ax)
filtered_ios_gpdf.plot(ax=ax)

This looks good in this small view, but looking at it in a map, we can see some weirdnesses at the end and at the curve on Miramonte

In [ ]:
import arrow

In [ ]:
new_android_gpdf["fmt_time"] = new_android_gpdf.ts.apply(lambda ts: arrow.get(ts).to("America/Los_Angeles"))

In [ ]:
import folium

In [ ]:
curr_map = folium.Map()
gt_leg_gj = sd_ucb.get_geojson_for_leg(gt_leg)
sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(zip(new_android_gpdf.longitude, new_android_gpdf.latitude))),properties={"style": {"color": "red"}, "ts": list(new_android_gpdf.ts)})
old_sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(zip(android_control_gpdf.longitude, android_control_gpdf.latitude))),properties={"style": {"color": "yellow"}, "ts": list(android_control_gpdf.ts)})
filtered_sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(zip(filtered_android_gpdf.longitude, filtered_android_gpdf.latitude))),properties={"style": {"color": "blue"}, "ts": list(filtered_android_gpdf.ts)})
gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj[2], name="ground_truth_points", color="green")
sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values")
old_sensed_leg_gj_feature = folium.GeoJson(old_sensed_section_gj, name="old sensed_values")
filtered_sensed_leg_gj_feature = folium.GeoJson(filtered_sensed_section_gj, name="filtered sensed_values")
sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="sensed_points", color="red", tz="America/Los_Angeles")
old_sensed_leg_gj_points = ezpv.get_point_markers(old_sensed_section_gj, name="old_sensed_points", color="yellow", tz="America/Los_Angeles")
filtered_sensed_leg_gj_points = ezpv.get_point_markers(filtered_sensed_section_gj, name="filtered_sensed_points", color="blue", tz="America/Los_Angeles")
curr_map.add_child(gt_leg_gj_feature)
curr_map.add_child(gt_leg_gj_points)
curr_map.add_child(sensed_leg_gj_feature)
curr_map.add_child(sensed_leg_gj_points)
curr_map.add_child(old_sensed_leg_gj_feature)
curr_map.add_child(old_sensed_leg_gj_points)
curr_map.add_child(filtered_sensed_leg_gj_feature)
curr_map.add_child(filtered_sensed_leg_gj_points)
curr_map.fit_bounds(sensed_leg_gj_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map

####  `shapely.interp1d`: Hard case (android locations are just really messed up)

- Works poorly with tolerance = 2m (25% valid ratio)
- Increasing tolerance to 5m bumps up the valid ratio to 50%, but then it introduces points that are not really close enough to ground truth (e.g. around loc 219)
- Increasing tolerance to 10m bumps up the valid ratio to 63%, but then it makes the ground truth even worse (e.g. around loc 250-268 and 390-393)

In [ ]:
android_trip = pv_la.map()["android"]["ucb-sdb-android-1"]["evaluation_ranges"][0]["evaluation_trip_ranges"][0]
ios_trip = pv_la.map()["ios"]["ucb-sdb-ios-1"]["evaluation_ranges"][0]["evaluation_trip_ranges"][0]
android_accuracy_df = android_trip["location_df"]
ios_accuracy_df = ios_trip["location_df"]

In [ ]:
android_control_gpdf = gpd.GeoDataFrame(
        android_accuracy_df, geometry=android_accuracy_df.apply(lambda lr: shp.geometry.Point(lr.longitude, lr.latitude), axis=1))
ios_control_gpdf = gpd.GeoDataFrame(
        ios_accuracy_df, geometry=ios_accuracy_df.apply(lambda lr: shp.geometry.Point(lr.longitude, lr.latitude), axis=1))

In [ ]:
len(android_control_gpdf), len(ios_control_gpdf)

In [ ]:
import scipy.interpolate as sci

In [ ]:
android_lat_fn = sci.interp1d(x=android_control_gpdf.ts, y=android_control_gpdf.latitude, fill_value="extrapolate")
android_lon_fn = sci.interp1d(x=android_control_gpdf.ts, y=android_control_gpdf.longitude, fill_value="extrapolate")
ios_lat_fn = sci.interp1d(x=ios_control_gpdf.ts, y=ios_control_gpdf.latitude, fill_value="extrapolate")
ios_lon_fn = sci.interp1d(x=ios_control_gpdf.ts, y=ios_control_gpdf.longitude, fill_value="extrapolate")

In [ ]:
new_ts_range = np.arange(android_trip["start_ts"], android_trip["end_ts"], 1)
new_fmt_time_range = [arrow.get(ts).to(sd_la.eval_tz) for ts in new_ts_range]

In [ ]:
new_android_lat = android_lat_fn(new_ts_range)
new_android_lng = android_lon_fn(new_ts_range)
new_ios_lat = ios_lat_fn(new_ts_range)
new_ios_lng = ios_lon_fn(new_ts_range)
new_android_gpdf = gpd.GeoDataFrame({
    "latitude": new_android_lat,
    "longitude": new_android_lng,
    "ts": new_ts_range,
    "fmt_time": new_fmt_time_range,
    "geometry": [shp.geometry.Point(x, y) for x, y in zip(new_android_lng, new_android_lat)]
})
new_ios_gpdf = gpd.GeoDataFrame({
    "latitude": new_ios_lat,
    "longitude": new_ios_lng,
    "ts": new_ts_range,
    "fmt_time": new_fmt_time_range,
    "geometry": [shp.geometry.Point(x, y) for x, y in zip(new_ios_lng, new_ios_lat)]
})

In [ ]:
new_android_gpdf[["fmt_time", "geometry"]].head(), new_ios_gpdf[["fmt_time", "geometry"]].head()

In [ ]:
distances = new_android_gpdf.distance(new_ios_gpdf) * (R/360); distances.head()

In [ ]:
valid_mask = distances <= 10; np.nonzero(np.logical_not(valid_mask))

In [ ]:
filtered_android_gpdf = new_android_gpdf[valid_mask]; filtered_android_gpdf.head()
filtered_ios_gpdf = new_ios_gpdf[valid_mask]; filtered_ios_gpdf.head()

In [ ]:
valid_ratio = len(filtered_android_gpdf)/len(new_android_gpdf)
assert valid_ratio == len(filtered_ios_gpdf)/len(new_ios_gpdf)
valid_ratio

In [ ]:
ifig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,15))
# android_control_gpdf.plot(ax=ax)
# ios_control_gpdf.plot(ax=ax)
# new_android_gpdf.plot(ax=ax)
# new_ios_gpdf.plot(ax=ax)
filtered_android_gpdf.plot(ax=ax)
filtered_ios_gpdf.plot(ax=ax)

This looks good in this small view, but looking at it in a map, we can see some weirdnesses at the end and at the curve on Miramonte

In [ ]:
import arrow

In [ ]:
new_android_gpdf["fmt_time"] = new_android_gpdf.ts.apply(lambda ts: arrow.get(ts).to("America/Los_Angeles"))

In [ ]:
import folium

In [ ]:
curr_map = folium.Map()
gt_leg_gj = sd_ucb.get_geojson_for_leg(gt_leg)
sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(zip(new_android_gpdf.longitude, new_android_gpdf.latitude))),properties={"style": {"color": "red"}, "ts": list(new_android_gpdf.ts)})
old_sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(zip(android_control_gpdf.longitude, android_control_gpdf.latitude))),properties={"style": {"color": "yellow"}, "ts": list(android_control_gpdf.ts)})
filtered_sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(zip(filtered_android_gpdf.longitude, filtered_android_gpdf.latitude))),properties={"style": {"color": "blue"}, "ts": list(filtered_android_gpdf.ts)})
gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj[2], name="ground_truth_points", color="green")
sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values")
old_sensed_leg_gj_feature = folium.GeoJson(old_sensed_section_gj, name="old sensed_values")
filtered_sensed_leg_gj_feature = folium.GeoJson(filtered_sensed_section_gj, name="filtered sensed_values")
sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="sensed_points", color="red", tz="America/Los_Angeles")
old_sensed_leg_gj_points = ezpv.get_point_markers(old_sensed_section_gj, name="old_sensed_points", color="yellow", tz="America/Los_Angeles")
filtered_sensed_leg_gj_points = ezpv.get_point_markers(filtered_sensed_section_gj, name="filtered_sensed_points", color="blue", tz="America/Los_Angeles")
curr_map.add_child(gt_leg_gj_feature)
curr_map.add_child(gt_leg_gj_points)
curr_map.add_child(sensed_leg_gj_feature)
curr_map.add_child(sensed_leg_gj_points)
curr_map.add_child(old_sensed_leg_gj_feature)
curr_map.add_child(old_sensed_leg_gj_points)
curr_map.add_child(filtered_sensed_leg_gj_feature)
curr_map.add_child(filtered_sensed_leg_gj_points)
curr_map.fit_bounds(sensed_leg_gj_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map

####  `shapely.interp1d`: Temporal accuracy check

Now let's check this for the temporal accuracy motivating use case

In [ ]:
android_trip = pv_ucb.map()["android"]["ucb-sdb-android-1"]["evaluation_ranges"][0]["evaluation_trip_ranges"][0]["evaluation_section_ranges"][5]
ios_trip = pv_ucb.map()["ios"]["ucb-sdb-ios-1"]["evaluation_ranges"][0]["evaluation_trip_ranges"][0]["evaluation_section_ranges"][5]
android_accuracy_df = android_trip["location_df"]
ios_accuracy_df = ios_trip["location_df"]

In [ ]:
android_control_gpdf = gpd.GeoDataFrame(
        android_accuracy_df, geometry=android_accuracy_df.apply(lambda lr: shp.geometry.Point(lr.longitude, lr.latitude), axis=1))
ios_control_gpdf = gpd.GeoDataFrame(
        ios_accuracy_df, geometry=ios_accuracy_df.apply(lambda lr: shp.geometry.Point(lr.longitude, lr.latitude), axis=1))

In [ ]:
len(android_control_gpdf), len(ios_control_gpdf)

In [ ]:
import scipy.interpolate as sci

In [ ]:
android_lat_fn = sci.interp1d(x=android_control_gpdf.ts, y=android_control_gpdf.latitude, fill_value="extrapolate")
android_lon_fn = sci.interp1d(x=android_control_gpdf.ts, y=android_control_gpdf.longitude, fill_value="extrapolate")
ios_lat_fn = sci.interp1d(x=ios_control_gpdf.ts, y=ios_control_gpdf.latitude, fill_value="extrapolate")
ios_lon_fn = sci.interp1d(x=ios_control_gpdf.ts, y=ios_control_gpdf.longitude, fill_value="extrapolate")

In [ ]:
new_ts_range = np.arange(android_trip["start_ts"], android_trip["end_ts"], 1)
new_fmt_time_range = [arrow.get(ts).to(sd_la.eval_tz) for ts in new_ts_range]

In [ ]:
new_android_lat = android_lat_fn(new_ts_range)
new_android_lng = android_lon_fn(new_ts_range)
new_ios_lat = ios_lat_fn(new_ts_range)
new_ios_lng = ios_lon_fn(new_ts_range)
new_android_gpdf = gpd.GeoDataFrame({
    "latitude": new_android_lat,
    "longitude": new_android_lng,
    "ts": new_ts_range,
    "fmt_time": new_fmt_time_range,
    "geometry": [shp.geometry.Point(x, y) for x, y in zip(new_android_lng, new_android_lat)]
})
new_ios_gpdf = gpd.GeoDataFrame({
    "latitude": new_ios_lat,
    "longitude": new_ios_lng,
    "ts": new_ts_range,
    "fmt_time": new_fmt_time_range,
    "geometry": [shp.geometry.Point(x, y) for x, y in zip(new_ios_lng, new_ios_lat)]
})

In [ ]:
new_android_gpdf[["fmt_time", "geometry"]].head(), new_ios_gpdf[["fmt_time", "geometry"]].head()

In [ ]:
distances = new_android_gpdf.distance(new_ios_gpdf) * (R/360); distances.head()

In [ ]:
valid_mask = distances <= 10; np.nonzero(np.logical_not(valid_mask))

In [ ]:
filtered_android_gpdf = new_android_gpdf[valid_mask]; filtered_android_gpdf.head()
filtered_ios_gpdf = new_ios_gpdf[valid_mask]; filtered_ios_gpdf.head()

In [ ]:
valid_ratio = len(filtered_android_gpdf)/len(new_android_gpdf)
assert valid_ratio == len(filtered_ios_gpdf)/len(new_ios_gpdf)
valid_ratio

In [ ]:
ifig, ax_array = plt.subplots(nrows=1, ncols=2, figsize=(15,15))
android_control_gpdf.plot(ax=ax_array[0])
ios_control_gpdf.plot(ax=ax_array[0])
# new_android_gpdf.plot(ax=ax)
# new_ios_gpdf.plot(ax=ax)
filtered_android_gpdf.plot(ax=ax_array[1])
filtered_ios_gpdf.plot(ax=ax_array[1])

This looks good in this small view, but looking at it in a map, we can see some weirdnesses at the end and at the curve on Miramonte

In [ ]:
import arrow

In [ ]:
new_android_gpdf["fmt_time"] = new_android_gpdf.ts.apply(lambda ts: arrow.get(ts).to("America/Los_Angeles"))

In [ ]:
import folium

In [ ]:
curr_map = folium.Map()
gt_leg_gj = sd_ucb.get_geojson_for_leg(gt_leg)
# sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(zip(new_android_gpdf.longitude, new_android_gpdf.latitude))),properties={"style": {"color": "red"}, "ts": list(new_android_gpdf.ts)})
old_sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(zip(android_control_gpdf.longitude, android_control_gpdf.latitude))),properties={"style": {"color": "yellow"}, "ts": list(android_control_gpdf.ts)})
oldi_sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(zip(ios_control_gpdf.longitude, ios_control_gpdf.latitude))),properties={"style": {"color": "red"}, "ts": list(ios_control_gpdf.ts)})
filtered_sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(zip(filtered_android_gpdf.longitude, filtered_android_gpdf.latitude))),properties={"style": {"color": "blue"}, "ts": list(filtered_android_gpdf.ts)})
gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj[2], name="ground_truth_points", color="green")
# sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values")
old_sensed_leg_gj_feature = folium.GeoJson(old_sensed_section_gj, name="sensed_values (android)")
oldi_sensed_leg_gj_feature = folium.GeoJson(oldi_sensed_section_gj, name="sensed_values (ios)")
filtered_sensed_leg_gj_feature = folium.GeoJson(filtered_sensed_section_gj, name="filtered sensed_values")
# sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="sensed_points", color="red", tz="America/Los_Angeles")
old_sensed_leg_gj_points = ezpv.get_point_markers(old_sensed_section_gj, name="sensed_points (android)", color="yellow", tz="America/Los_Angeles")
oldi_sensed_leg_gj_points = ezpv.get_point_markers(oldi_sensed_section_gj, name="sensed_points (ios)", color="red", tz="America/Los_Angeles")
filtered_sensed_leg_gj_points = ezpv.get_point_markers(filtered_sensed_section_gj, name="filtered_sensed_points", color="blue", tz="America/Los_Angeles")
curr_map.add_child(gt_leg_gj_feature)
curr_map.add_child(gt_leg_gj_points)
# curr_map.add_child(sensed_leg_gj_feature)
# curr_map.add_child(sensed_leg_gj_points)
curr_map.add_child(old_sensed_leg_gj_feature)
curr_map.add_child(old_sensed_leg_gj_points)
curr_map.add_child(oldi_sensed_leg_gj_feature)
curr_map.add_child(oldi_sensed_leg_gj_points)
curr_map.add_child(filtered_sensed_leg_gj_feature)
curr_map.add_child(filtered_sensed_leg_gj_points)
curr_map.fit_bounds(sensed_leg_gj_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map

In [ ]:
android_control_gpdf.iloc[-1].ts, android_control_gpdf.iloc[-1].ts + 60 * 60

In [ ]:
pv_ucb.map()["android"]["ucb-sdb-android-1"]["evaluation_ranges"][0]["location_df"].query("ts > 1563986933 & ts < 1563990533")

In [ ]:
new_android_gpdf.loc[71:83]

In [ ]:
def get_polygons(pvunp):
    """
    Return GeoSeries of polygons
    """
    polygons = []
    trips = pvunp.spec_details.curr_spec['evaluation_trips']

    for trip in trips:
        for leg in trip['legs']:
            if 'loc' in leg and leg['loc']['geometry']['type'] == 'Polygon':
                polygons.append(shp.geometry.Polygon(leg['loc']['geometry']['coordinates'][0]))
            if 'end_loc' in leg and leg['end_loc']['geometry']['type'] == 'Polygon':
                polygons.append(shp.geometry.Polygon(leg['end_loc']['geometry']['coordinates'][0]))
            if 'start_loc' in leg and leg['start_loc']['geometry']['type'] == 'Polygon':
                polygons.append(shp.geometry.Polygon(leg['start_loc']['geometry']['coordinates'][0]))
    return gpd.GeoSeries(polygons)

In [ ]:
def is_point_outside_polygons(loc_row, polygons):
    """
    Utility function to check if a point represented by a row in a location dataframe
    is contained within a series of Shapely polygons
    """
    # print(loc_row)
    point = loc_row.geometry
    inside_polygons = polygons.contains(point)
    return not inside_polygons.any()

def get_travel_trajectory(df, polygons):
    """ 
    Filters the dataframe of location points to only include values outside the defined polygons
    """
    geo_df = gpd.GeoDataFrame(
        df, geometry=df.apply(lambda lr: shp.geometry.Point(lr.longitude, lr.latitude), axis=1))
    geo_df["outside_polygons"] = geo_df.apply(lambda r: is_point_outside_polygons(r, polygons), axis=1)
    # return a slice instead of setting a column value
    return geo_df.query("outside_polygons==True")

def get_gt_linestring(gt_leg):
    """
    Get lat-long corrdinates in ground truth
    """
    if 'route_coords' in gt_leg:
        coords = gt_leg['route_coords']['geometry']['coordinates']
    else:
        coords = []
    return shp.geometry.LineString(coords)

def get_measured_points(df, polygons):
    """
    Get lat-long coordinates recorded from phone
    """
    return get_travel_trajectory(df, polygons)

def get_projection(loc_row, line):
    point = geometry.Point(loc_row.longitude, loc_row.latitude)
    return point.distance(line)

def get_shortest_distances(df, line):
    """
    Returns a series representing the distances from each point in a location dataframe to a line
    """
    return df.distance(line)

def convert_to_xy(points):
    xy_points = []
    R = 6371000 # Radius of the earth in m
    
    for point in points:
        lon = point[0]
        lat = point[1]
        
        theta = math.pi/2 - math.radians(lat) 
        phi = math.radians(lon)
        
        x = R * math.sin(theta) * math.cos(phi)
        y = R * math.sin(theta) * math.sin(phi)
        xy_points.append([x, y])
        
    return xy_points

In [ ]:
def get_spatial_errors(pvunp):
    spatial_error_list = []
    # This is a GeoSeries
    polygons = get_polygons(pvunp)
    
    for phone_os, phone_map in pvunp.map().items():
        for phone_label, phone_detail_map in phone_map.items():
            for (r_idx, r) in enumerate(phone_detail_map["evaluation_ranges"]):
                run_errors = []
                for (tr_idx, tr) in enumerate(r["evaluation_trip_ranges"]):
                    trip_errors = []
                    for (sr_idx, sr) in enumerate(tr["evaluation_section_ranges"]):
                        # This is a Shapely LineString
                        section_gt_leg = pvunp.spec_details.get_ground_truth_for_leg(tr["trip_id_base"], sr["trip_id_base"])
                        section_gt_points = get_gt_linestring(section_gt_leg)
                        if section_gt_points.is_empty:
                            print("No ground truth route for %s %s, must be polygon, skipping..." % (tr["trip_id_base"], sr["trip_id_base"]))
                            assert section_gt_leg["type"] != "TRAVEL", "For %s, %s, %s, %s, %s found type %s" % (phone_os, phone_label, r_idx, tr_idx, sr_idx, section_gt_leg["type"])
                            continue
                        if len(sr['location_df']) == 0:
                            print("No sensed locations found, role = %s skipping..." % (r["eval_role_base"]))
                            # assert r["eval_role_base"] == "power_control", "Found no locations for %s, %s, %s, %s, %s" % (phone_os, phone_label, r_idx, tr_idx, sr_idx)
                            continue

                        # This is a GeoDataFrame
                        section_measured_points = get_measured_points(sr['location_df'], polygons)
                        line = section_gt_points
                        degree_dist = get_shortest_distances(section_measured_points.geometry, line)
                        meter_dist = degree_dist * (6371000/360) # Radius of the earth in m
                        spatial_error_entries = [{"phone_os": phone_os, "phone_label": phone_label,
                                               "timeline": pvunp.spec_details.curr_spec["id"],
                                               "run": r_idx, "role": r["eval_role_base"],
                                               "trip_id": tr["trip_id_base"], "section_id": sr["trip_id_base"],
                                               "error": e} for e in meter_dist]
                        spatial_error_list.extend(spatial_error_entries)
    return spatial_error_list

In [ ]:
spatial_errors_list = []
spatial_errors_list.extend(get_spatial_errors(pv_la))
spatial_errors_list.extend(get_spatial_errors(pv_sj))
spatial_errors_list.extend(get_spatial_errors(pv_ucb))

spatial_errors_df = pd.DataFrame(spatial_errors_list)

In [ ]:
r2q_map = {"power_control": 0, "HAMFDC": 1, "MAHFDC": 2, "HAHFDC": 3, "accuracy_control": 4}
q2r_map = {0: "power", 1: "HAMFDC", 2: "MAHFDC", 3: "HAHFDC", 4: "accuracy"}

In [ ]:
spatial_errors_df["quality"] = spatial_errors_df.role.apply(lambda r: r2q_map[r])
spatial_errors_df["label"] = spatial_errors_df.role.apply(lambda r: r.replace('_control', ''))
timeline_list = ["train_bus_ebike_mtv_ucb", "car_scooter_brex_san_jose", "unimodal_trip_car_bike_mtv_la"]

In [ ]:
spatial_errors_df.head()

In [ ]:
spatial_errors_df.query('timeline == "train_bus_ebike_mtv_ucb" & run == 0 & phone_os == "android" & quality == 0').error

In [ ]:
ifig, ax_array = plt.subplots(nrows=2,ncols=3,figsize=(12,6), sharex=False, sharey=False)
timeline_list = ["train_bus_ebike_mtv_ucb", "car_scooter_brex_san_jose", "unimodal_trip_car_bike_mtv_la"]
for i, tl in enumerate(timeline_list):
    spatial_errors_df.query("timeline == @tl & phone_os == 'android' & quality > 0").boxplot(ax = ax_array[0][i], column=["error"], by=["quality"])
    ax_array[0][i].set_title(tl)
    spatial_errors_df.query("timeline == @tl & phone_os == 'ios' & quality > 0").boxplot(ax = ax_array[1][i], column=["error"], by=["quality"])
    ax_array[1][i].set_title("")

for i, ax in enumerate(ax_array[0]):
    ax.set_xticklabels([q2r_map[int(t.get_text())] for t in ax.get_xticklabels()])
    ax.set_xlabel("")

for i, ax in enumerate(ax_array[1]):
    ax.set_xticklabels([q2r_map[int(t.get_text())] for t in ax.get_xticklabels()])
    ax.set_xlabel("")

ax_array[0][0].set_ylabel("Battery drain (android)")
ax_array[1][0].set_ylabel("Battery drain (iOS)")
ifig.suptitle("Power v/s quality over multiple timelines")
# ifig.tight_layout()

In [ ]:
ifig, ax_array = plt.subplots(nrows=2,ncols=4,figsize=(25,10), sharex=True, sharey=True)
timeline_list = ["train_bus_ebike_mtv_ucb"]
for i, tl in enumerate(timeline_list):
    for q in range(1,5):
        sel_df = spatial_errors_df.query("timeline == @tl & phone_os == 'android' & quality == @q")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*i][q-1], column=["error"], by=["section_id"])
        ax_array[2*i][q-1].tick_params(axis="x", labelrotation=45)
        sel_df = spatial_errors_df.query("timeline == @tl & phone_os == 'ios' & quality == @q")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*i+1][q-1], column=["error"], by=["section_id"])
#        ax_array[i][].set_title("")

def make_acronym(s):
    ssl = s.split("_")
    # print("After splitting %s, we get %s" % (s, ssl))
    if len(ssl) == 0 or len(ssl[0]) == 0:
        return ""
    else:
        return "".join([ss[0] for ss in ssl])

for q in range(1,5):
    ax_array[0][q-1].set_title(q2r_map[q])
    curr_ticks = [t.get_text() for t in ax_array[1][q-1].get_xticklabels()]
    new_ticks = [make_acronym(t) for t in curr_ticks]
    ax_array[1][q-1].set_xticklabels(new_ticks)
    
print(list(zip(curr_ticks, new_ticks)))
# fig.text(0,0,"%s"% list(zip(curr_ticks, new_ticks)))

In [ ]:
timeline_list = ["train_bus_ebike_mtv_ucb"]
for i, tl in enumerate(timeline_list):
    unique_sections = spatial_errors_df.query("timeline == @tl").section_id.unique()
    ifig, ax_array = plt.subplots(nrows=2,ncols=len(unique_sections),figsize=(40,10), sharex=True, sharey=False)
    for sid, s_name in enumerate(unique_sections):
        sel_df = spatial_errors_df.query("timeline == @tl & phone_os == 'android' & section_id == @s_name & quality > 0")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*i][sid], column=["error"], by=["quality"])
        ax_array[2*i][sid].set_title(s_name)
        sel_df = spatial_errors_df.query("timeline == @tl & phone_os == 'ios' & section_id == @s_name & quality > 0")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*i+1][sid], column=["error"], by=["quality"])
        ax_array[2*i+1][sid].set_title("")
#        ax_array[i][].set_title("")


### Focus only on sections where the max error is > 100 meters

In [ ]:
timeline_list = ["train_bus_ebike_mtv_ucb"]
for i, tl in enumerate(timeline_list):
    unique_sections = pd.Series(spatial_errors_df.query("timeline == @tl").section_id.unique())
    sections_with_outliers_mask = unique_sections.apply(lambda s_name: spatial_errors_df.query("timeline == 'train_bus_ebike_mtv_ucb' & section_id == @s_name").error.max() > 100)
    sections_with_outliers = unique_sections[sections_with_outliers_mask]   
    ifig, ax_array = plt.subplots(nrows=2,ncols=len(sections_with_outliers),figsize=(16,4), sharex=True, sharey=False)
    for sid, s_name in enumerate(sections_with_outliers):
        sel_df = spatial_errors_df.query("timeline == @tl & phone_os == 'android' & section_id == @s_name & quality > 0")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*i][sid], column=["error"], by=["quality"])
        ax_array[2*i][sid].set_title(s_name)
        sel_df = spatial_errors_df.query("timeline == @tl & phone_os == 'ios' & section_id == @s_name & quality > 0")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*i+1][sid], column=["error"], by=["quality"])
        ax_array[2*i+1][sid].set_title("")
#        ax_array[i][].set_title("")
    ifig.suptitle("")

### Validation of outliers

#### (express bus iOS, MAHFDC)

ok, so it looks like the error is non-trivial across all runs, but run #1 is the worst and is responsible for the majority of the outliers. And this is borne out by the map, where on run #1, we end up with points in San Leandro!!

In [ ]:
spatial_errors_df.query("phone_os == 'ios' & quality == 2 & section_id == 'express_bus' & error > 500").run.unique()

In [ ]:
spatial_errors_df.query("phone_os == 'ios' & quality == 2 & section_id == 'express_bus'").boxplot(column="error", by="run")

In [ ]:
ucb_and_back = pv_ucb.map()["ios"]["ucb-sdb-ios-3"]["evaluation_ranges"][1]; ucb_and_back["trip_id"]

In [ ]:
back_trip = ucb_and_back["evaluation_trip_ranges"][2]; print(back_trip["trip_id"])

In [ ]:
eb_leg = back_trip["evaluation_section_ranges"][4]; print(eb_leg["trip_id"])

In [ ]:
gt_leg = sd_ucb.get_ground_truth_for_leg(back_trip["trip_id_base"], eb_leg["trip_id_base"]); gt_leg["id"]

In [ ]:
import folium

In [ ]:
curr_map = folium.Map()
gt_leg_gj = sd_ucb.get_geojson_for_leg(gt_leg)
sensed_section_gj = ezpv.get_geojson_for_leg(eb_leg)
gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj[2], name="ground_truth_points", color="green")
sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values")
sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="sensed_points", color="red", tz="America/Los_Angeles")
curr_map.add_child(gt_leg_gj_feature)
curr_map.add_child(gt_leg_gj_points)
curr_map.add_child(sensed_leg_gj_feature)
curr_map.add_child(sensed_leg_gj_points)
curr_map.fit_bounds(sensed_leg_gj_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map

In [ ]:
curr_map = folium.Map()
gt_leg_gj = sd_ucb.get_geojson_for_leg(gt_leg)
gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj[2], name="ground_truth_points", color="green")

colors = ["red", "yellow", "blue"]
for run in range(3):
    ucb_and_back = pv_ucb.map()["ios"]["ucb-sdb-ios-3"]["evaluation_ranges"][run]; ucb_and_back["trip_id"]
    back_trip = ucb_and_back["evaluation_trip_ranges"][2]; print(back_trip["trip_id"])
    eb_leg = back_trip["evaluation_section_ranges"][4]; print(eb_leg["trip_id"])
    sensed_section_gj = ezpv.get_geojson_for_leg(eb_leg)
    sensed_section_gj["properties"]["style"]["color"] = colors[run]
    sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="run %s" % run)
    sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="points for %d" % run, color=colors[run], tz="America/Los_Angeles")
    curr_map.add_child(sensed_leg_gj_feature)
    curr_map.add_child(sensed_leg_gj_points)
    
curr_map.add_child(gt_leg_gj_feature)
curr_map.add_child(gt_leg_gj_points)
curr_map.fit_bounds(gt_leg_gj_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map

#### (commuter rail aboveground android, HAMFDC)

ok, so it looks like the error is non-trivial across all runs, but run #1 is the worst and is responsible for the majority of the outliers. And this is borne out by the map, where on run #1, we end up with points in San Leandro!!

In [ ]:
spatial_errors_df.query("phone_os == 'android' & quality == 1 & section_id == 'commuter_rail_aboveground' & error > 500").run.unique()

In [ ]:
ucb_and_back = pv_ucb.map()["android"]["ucb-sdb-android-3"]["evaluation_ranges"][0]; ucb_and_back["trip_id"]
to_trip = ucb_and_back["evaluation_trip_ranges"][0]; print(to_trip["trip_id"])
train_leg = to_trip["evaluation_section_ranges"][2]; print(train_leg["trip_id"])
gt_leg = sd_ucb.get_ground_truth_for_leg(to_trip["trip_id_base"], train_leg["trip_id_base"]); gt_leg["id"]

In [ ]:
curr_map = folium.Map()
gt_leg_gj = sd_ucb.get_geojson_for_leg(gt_leg)
sensed_section_gj = ezpv.get_geojson_for_leg(train_leg)
gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj[2], name="ground_truth_points", color="green")
sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values")
sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="sensed_points", color="red", tz="America/Los_Angeles")
curr_map.add_child(gt_leg_gj_feature)
curr_map.add_child(gt_leg_gj_points)
curr_map.add_child(sensed_leg_gj_feature)
curr_map.add_child(sensed_leg_gj_points)
curr_map.fit_bounds(sensed_leg_gj_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map

In [ ]:
unique_sections = pd.Series(spatial_errors_df.query("timeline == 'train_bus_ebike_mtv_ucb'").section_id.unique())
sections_with_outliers_mask = unique_sections.apply(lambda s_name: spatial_errors_df.query("timeline == 'train_bus_ebike_mtv_ucb' & section_id == @s_name").error.max() > 100)
unique_sections[sections_with_outliers_mask]

#### (walk_to_bus android, HAMFDC, HAHFDC)

ok, so it looks like the error is non-trivial across all runs, but run #1 is the worst and is responsible for the majority of the outliers. And this is borne out by the map, where on run #1, we end up with points in San Leandro!!

In [ ]:
spatial_errors_df.query("phone_os == 'android' & (quality == 1 | quality == 3) & section_id == 'walk_to_bus' & error > 500").run.unique()

In [ ]:
spatial_errors_df.query("phone_os == 'android' & (quality == 1 | quality == 3) & section_id == 'walk_to_bus' & error > 500")

In [ ]:
spatial_errors_df.query("phone_os == 'android' & (quality == 1 | quality == 3) & section_id == 'walk_to_bus'").boxplot(column="error", by="run")

In [ ]:
ucb_and_back = pv_ucb.map()["android"]["ucb-sdb-android-2"]["evaluation_ranges"][0]; ucb_and_back["trip_id"]
to_trip = ucb_and_back["evaluation_trip_ranges"][0]; print(to_trip["trip_id"])
wb_leg = to_trip["evaluation_section_ranges"][6]; print(wb_leg["trip_id"])
gt_leg = sd_ucb.get_ground_truth_for_leg(to_trip["trip_id_base"], wb_leg["trip_id_base"]); gt_leg["id"]

In [ ]:
import folium

In [ ]:
curr_map = folium.Map()
gt_leg_gj = sd_ucb.get_geojson_for_leg(gt_leg)
sensed_section_gj = ezpv.get_geojson_for_leg(wb_leg)
gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj[2], name="ground_truth_points", color="green")
sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values")
sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="sensed_points", color="red", tz="America/Los_Angeles")
curr_map.add_child(gt_leg_gj_feature)
curr_map.add_child(gt_leg_gj_points)
curr_map.add_child(sensed_leg_gj_feature)
curr_map.add_child(sensed_leg_gj_points)
curr_map.fit_bounds(sensed_leg_gj_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map

#### (light_rail_below_above_ground, android, accuracy_control)

ok, so it looks like the error is non-trivial across all runs, but run #1 is the worst and is responsible for the majority of the outliers. And this is borne out by the map, where on run #1, we end up with points in San Leandro!!

In [ ]:
spatial_errors_df.query("phone_os == 'android' & quality == 4 & section_id == 'light_rail_below_above_ground' & error > 100").run.unique()

In [ ]:
spatial_errors_df.query("phone_os == 'android' & (quality == 4) & section_id == 'light_rail_below_above_ground'").boxplot(column="error", by="run")

In [ ]:
ucb_and_back = pv_ucb.map()["android"]["ucb-sdb-android-2"]["evaluation_ranges"][0]; ucb_and_back["trip_id"]
back_trip = ucb_and_back["evaluation_trip_ranges"][2]; print(back_trip["trip_id"])
lt_leg = back_trip["evaluation_section_ranges"][7]; print(lt_leg["trip_id"])
gt_leg = sd_ucb.get_ground_truth_for_leg(back_trip["trip_id_base"], lt_leg["trip_id_base"]); gt_leg["id"]

In [ ]:
import folium

In [ ]:
curr_map = folium.Map()
gt_leg_gj = sd_ucb.get_geojson_for_leg(gt_leg)
gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj[2], name="ground_truth_points", color="green")

colors = ["red", "yellow", "blue"]
for run in range(3):
    ucb_and_back = pv_ucb.map()["android"]["ucb-sdb-android-2"]["evaluation_ranges"][run]; ucb_and_back["trip_id"]
    back_trip = ucb_and_back["evaluation_trip_ranges"][2]; print(back_trip["trip_id"])
    lt_leg = back_trip["evaluation_section_ranges"][7]; print(lt_leg["trip_id"])
    sensed_section_gj = ezpv.get_geojson_for_leg(lt_leg)
    sensed_section_gj["properties"]["style"]["color"] = colors[run]
    sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="run %s" % run)
    sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="points for %d" % run, color=colors[run], tz="America/Los_Angeles")
    curr_map.add_child(sensed_leg_gj_feature)
    curr_map.add_child(sensed_leg_gj_points)
    
curr_map.add_child(gt_leg_gj_feature)
curr_map.add_child(gt_leg_gj_points)
curr_map.fit_bounds(gt_leg_gj_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map